## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

-
-

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

# Алгоритм Гаффмана

В цьому алгоритмі доцільно імплементувати клас node та додаткові функції в Huffman для побудови дерева кодування

In [1]:
class Node:
    def __init__(self, weight: int, chars: str):
        self.weight = weight
        self.chars = chars

    def __lt__(self, other):
        return self.weight < other.weight
    
    def __add__(self, other):
        return Node(self.weight + other.weight, self.chars + other.chars)
    
    def __iter__(self):
        return iter(self.chars)

class Huffman:
    @staticmethod
    def count_frequency(text: str) -> dict[str, int]:
        frequency = {}
        for char in text:
            frequency.setdefault(char, 0)
            frequency[char] += 1
        return frequency

    def encode(self, text: str) -> tuple[str, dict[str, str]]:
        frequency = self.count_frequency(text)

        tree = [Node(weight, [[char,'']]) for char, weight in frequency.items()]

        while len(tree) > 1:
            first_min=tree.pop(tree.index(min(tree)))
            second_min=tree.pop(tree.index(min(tree)))
            for char in first_min:
                char[1] = '0' + char[1]
            for char in second_min:
                char[1] = '1' + char[1]
            tree.append(first_min+second_min)

        huffman_code = {item[0]: item[1] for item in tree[0]}

        encoded_text = ''.join(huffman_code[char] for char in text)

        return encoded_text, huffman_code

    def decode(self, code: str, coding_dict: dict[str, str]) -> str:
        reverse=dict((v,k) for k,v in coding_dict.items())
        decoded_text = ""
        temp = ""
        for digit in code:
            temp += digit
            if temp in reverse:
                decoded_text += reverse[temp]
                temp = ""

        return decoded_text
    

if __name__ == "__main__":
    huffman = Huffman()
    text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
    encoded_text, coding_dict = huffman.encode(text)
    print(f"Original text: {text}")
    print(f"Encoded text: {encoded_text}")
    print(f"Coding dictionary: {coding_dict}")
    print(f"Decoded text: {huffman.decode(encoded_text, coding_dict)}")
    print(f'Bit length of original text: {len(text) * 8}')
    print(f'Bit length of encoded text: {len(encoded_text)}')
    print(f'Ratio of encoded text to original text: {len(encoded_text) / (len(text) * 8)}')
    assert huffman.decode(encoded_text, coding_dict) == text

Original text: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
Encoded text: 011011100100001001111101111100000011111100011110111110111011000001010000100110111000001010110100110111111110100110001110101101000010111100111110110101011111010011101001101001111010000011100011100101100000101011001011011110010000101000110011001101110111111110111011101100011011110000111111001011110001110111010101111101110011110000100110000010110110000111010001110101110101101011001111010110001010010110011100001001111110111110101101110110000010100001001111110101111001011001001011001110100100100000011010111100100110011001101

# Алгоритм LZW

In [ ]:
class LZW:
    def encode(self, text: str) -> tuple[str, list]:
        pass

    def decode(self, code: str, coding_dict: list) -> str:
        pass

# Алгоритм LZ77

Потрібно заміряти розміри саме тексту, проте для роботи доцільно використовувати список тюплів, тому для зручності варто імплементувати додаткові алгоритми _text2list та _list2text

In [ ]:
class LZ77:
    def __init__(self, buffer_size: int):
        pass

    def encode(self, text: str) -> str:
        pass

    def decode(self, code: str) -> str:
        pass

# Алгоритм Deflate

In [ ]:
class Deflate:
    def __init__(self, buffer_size: int):
        pass

    def encode(self, text: str) -> str:
        pass

    def decode(self, code: str) -> str:
        pass